<a href="https://colab.research.google.com/github/machine-perception-robotics-group/MPRGDeepLearningLectureNotebook/blob/master/14_rl/05_Deep_Q_Network_application.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 目的
強化学習におけるエージェントや環境設計(報酬， 行動, etc.)の重要さを理解する．

## 問題設定

Deep Q-Networkを用いて，波形の振幅/周波数/波長を制御し目標のsin波にフィッティングさせるAIを作成してみます．
波形調整する環境を自作し，エージェントや環境の振る舞い(報酬，行動など）を設計する．


**Deep Q-Network (DQN)**

> Deep Q-Network (DQN)は，GoogleのDeepmindが2016年に発表した手法で，Q学習におけるQテーブルを用いた行動価値の導出を，DCNNを用いた近似関数で代用するのが主な手法の内容です．
Q学習では，全ての状態と行動の組み合わせについて，行動価値をQテーブルに記録している．
そのため，状態数と行動数の組み合わせが膨大な環境に対して，膨大なメモリが必要となる問題を抱えていました．
この問題に対して，DQNはQテーブルそのものをDCNNで代用することにより，上記問題を解決しました．\
また，DQNにはその他にも強化学習における学習の安定性獲得のために，Experience replay，Target Q-Network，Reward clippingなどの工夫がなされている．


<img src="https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/2708544/0d5711a1-803f-f142-125e-6b70f84a3c77.png" width = 50%>




## 準備

### Google Colaboratoryの設定確認・変更
本チュートリアルではPyTorchを利用してニューラルネットワークの実装を確認，学習および評価を行います．
**GPUを用いて処理を行うために，上部のメニューバーの「ランタイム」→「ランタイムのタイプを変更」からハードウェアアクセラレータをGPUにしてください．**


### モジュールの追加インストール
下記のプログラムを実行して，実験結果の表示に必要な追加ライブラリやモジュールをインストールする．

In [ ]:
!apt update && apt install xvfb
!pip -q install pyglet
!pip -q install pyopengl
!pip -q install pyvirtualdisplay

## モジュールのインポート
はじめに必要なモジュールをインポートする．

In [ ]:
import numpy as np
import time
import datetime
import random
import cv2
import matplotlib.pyplot as plt
import collections
%matplotlib inline

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T

import warnings
warnings.simplefilter('ignore')

# GPUの確認
use_cuda = torch.cuda.is_available()
print('Use CUDA:', use_cuda)

## シード値の固定

In [ ]:
seed = 123
# Python random
random.seed(seed)
# Numpy
np.random.seed(seed)
# Pytorch
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.use_deterministic_algorithms = True

## 自作強化学習環境の定義

タスクの目的：波形を制御し，目標の波形へフィッティングさせること

制御波形の振幅/周波数/波長を制御し，目標波形(sin波)に適合させる環境を自作する．

**WAVEクラス**

制御波形を定義/パラメータ更新するためのクラスです．

In [ ]:
# 波形クラス
class WAVE():
  def __init__(self, x):
    """
    波形の初期設定. 各パラメータの上下限を設定.
    """
    self.x = x # x情報

    # 振幅
    self.A = 0
    self.range_A = [0.1, 2.0]
    # 周期
    self.T = 0
    self.range_T = [0.1, 3.0]
    # 周波数
    self.f = 0
    # 波長
    self.lam = 0
    self.range_lam = [0.1, 3.0]

  def set_seed(self, n_seed):
    """
    seed値の設定．
    """
    random.seed(n_seed)
    np.random.seed(n_seed)
    return

  def init_wave(self):
    """
    制御波形の初期化．
    """
    # 制御波形の初期パラメータ (ランダムver)
    #self.A = round(random.uniform(self.range_A[0], self.range_A[1]), 1)
    #self.T = round(random.uniform(self.range_T[0], self.range_T[1]), 1)
    #self.f = 1.0/self.T
    #self.lam = round(random.uniform(self.range_lam[0], self.range_lam[1]), 1)

    # 制御波形の初期パラメータ (固定ver)
    self.A = 2.0
    self.T = 1.0
    self.f = 1.0/self.T
    self.lam = 1.0

    y = self.A * np.sin(2 * np.pi * (self.f - self.x/self.lam))
    return y.astype(float)

  def checkparam(self):
    """
    initで設定した各パラメータの上下限をもとに各パラメータが設定値を大幅に超えないように制限．
    """
    # 振幅チェック
    if self.A < self.range_A[0]: self.A = self.range_A[0]
    elif self.A > self.range_A[1]: self.A = self.range_A[1]
    # 周期チェック
    if self.T < self.range_T[0]: self.T = self.range_T[0]
    elif self.T > self.range_T[1]: self.T = self.range_T[1]
    # 波長チェック
    if self.lam < self.range_lam[0]: self.lam = self.range_lam[0]
    elif self.lam > self.range_lam[1]: self.lam = self.range_lam[1]
    return

  def update(self, x):
    """
    現パラメータにおける波形を生成
    """
    self.f = 1.0/self.T
    y = self.A * np.sin(2.0 * np.pi * (self.f - x/self.lam))
    return y.astype(float)

**ENVクラス**

環境を定義しているクラスです． 上記のWAVEクラスはここで使用します．

In [ ]:
# 環境クラス
class ENV():
  def __init__(self, episode_max_step=500, obs_range=100, obs_mode="num"):
    """
    環境の設定
    episode_max_step：　1エピソード間のstep数
    obs_range：何時刻分の観測値を返すか (観測値が数値情報時のみ)
    obs_mode：観測値を画像(image)として返すか数値情報(num)として返すか
    """
    self.obs_mode = obs_mode
    if self.obs_mode == "num":
      self.observation = np.zeros(obs_range*2) # fit_wave + target_wave
    elif self.obs_mode == "image":
      self.observation = np.zeros((1,84,84)) # image
    self.time_step = 0
    self.episode_max_step = episode_max_step

    # 選択可能な行動リスト [振幅，周期,波長]
    self.action_list = [
        [0.0, 0.0, 0.0],
        [0.0, 0.0, 0.1],
        [0.0, 0.1, 0.0],
        [0.0, 0.1, 0.1],
        [0.1, 0.0, 0.0],
        [0.1, 0.0, 0.1],
        [0.1, 0.1, 0.0],
        [0.1, 0.1, 0.1],
        [0.0, 0.0, -0.1],
        [0.0, -0.1, 0.0],
        [0.0, -0.1, -0.1],
        [-0.1, 0.0, 0.0],
        [-0.1, 0.0, -0.1],
        [-0.1, -0.1, 0.0],
        [-0.1, -0.1, -0.1]
      ]

    self.x = np.linspace(0, 6*np.pi, obs_range) # x情報

    # 目標波形：sin波
    self.target_wave = 1.0 * np.sin(2.0 * np.pi * (0.5 - self.x/2.0))

    # 制御波形の初期化
    self.fit_wave = WAVE(self.x)


  def action_space(self):
    """
    選択可能な行動数を返す
    """
    return len(self.action_list)

  def observation_space(self):
    """
    観測値のshapeを返す
    """
    if self.obs_mode == "num":
      return self.observation.shape[0]
    elif self.obs_mode == "image":
      return self.observation.shape
    else:
      return

  def seed(self, n_seed):
    """
    環境のseed値を設定
    """
    self.fit_wave.set_seed(n_seed)
    return

  def random_action(self):
    """
    ランダムに行動選択を実行
    """
    return random.randrange(len(self.action_list))


  def cal_reward(self, tgt_wave, fit_wave):
    """
    報酬を算出 (ユークリッド距離を用いた波形の類似度)
    """
    # fit_waveとtarget_waveのユークリッド距離を算出
    dist = 0.0
    for t in range(len(fit_wave)):
      dist += (tgt_wave[t] - fit_wave[t])**2
    dist = 1.0/(1.0+np.sqrt(dist))

    # 類似度が0.95より大きい場合：2.0
    # それ以外：類似度
    if dist > 0.95:
      reward = 2.0
    else:
      reward = dist

    return reward

  # step関数
  def step(self, act):
    """
    渡された行動をもとに環境を1step進める
    観測情報,報酬値，終了フラグを返す
    """
    self.time_step += 1

    # 行動リストから行動を選択し各パラメータを更新
    actions = self.action_list[act]
    self.fit_wave.A += actions[0]
    self.fit_wave.T += actions[1]
    self.fit_wave.lam += actions[2]

    # 各パラメータの上下限を超えていないか確認
    self.fit_wave.checkparam()

    # 現パラメータで波形を生成
    self.out = self.fit_wave.update(self.x)

    # 観測情報を生成 (数値モード：調整波形と目標波形の数値情報, 画像モード：各波形をグラフとして描画した画像)
    if self.obs_mode == "num":
      self.observation = np.array(np.concatenate([self.out, self.target_wave]))
    elif self.obs_mode == "image":
      self.observation, _ = self.render(mode="rgb_array")

    # 報酬値算出
    reward = self.cal_reward(self.target_wave, self.out)

    # 終了フラグ (step数がepisode_max_step以上となったらエピソード終了)
    if self.time_step >= self.episode_max_step:
      done = True
    else:
      done = False

    return self.observation, reward, done

  def reset(self):
    """
    環境のリセット
    """
    self.out = self.fit_wave.init_wave()
    if self.obs_mode == "num":
      obs = np.array(np.concatenate([self.out, self.target_wave]))
    elif self.obs_mode == "image":
      obs, _ = self.render(mode="rgb_array")
    self.time_step = 0
    return obs

  def render(self, mode="rendering"):
    """
    現環境の描画
    modeが"rendering"の場合：表示のみを行う
    modeが"rgb_array"の場合：描画結果のカラー画像とグレースケール画像を返す．表示は行わない
    """
    x_len, y_len = 500, 500
    canvas = (np.ones((y_len, x_len, 3)) * 255.0).astype(np.uint8)

    target_pts, fit_pts = [], []
    for idx in range(len(self.target_wave)):
      x = int(idx/len(self.target_wave) * x_len)
      target_y = int(((self.target_wave[idx]+self.fit_wave.range_A[1]) / (self.fit_wave.range_A[1]*2.0)) * y_len)
      fit_y = int(((self.out[idx]+self.fit_wave.range_A[1]) / (self.fit_wave.range_A[1]*2.0)) * y_len)
      target_pts.append((x, target_y))
      fit_pts.append((x, fit_y))
    target_pts, fit_pts = np.array(tuple(target_pts)), np.array(tuple(fit_pts))

    cv2.polylines(canvas, [target_pts], False, (0, 0, 255), thickness=2)
    cv2.polylines(canvas, [fit_pts], False, (255, 0, 0), thickness=2)

    img_color = cv2.cvtColor(canvas, cv2.COLOR_BGR2RGB)
    if mode == "rgb_array":
      img_gray = cv2.cvtColor(canvas, cv2.COLOR_BGR2GRAY)
      img_gray = cv2.resize(img_gray, dsize=(84,84))
      img_gray = img_gray[np.newaxis,:,:]
      return img_gray, img_color
    else:
      plt.imshow(img_color)
      plt.show()
      return

In [ ]:
# 環境の定義
env = ENV(obs_mode="num")
env.seed(seed)
# 環境の初期化
obs = env.reset()
print('observation space:', env.observation_space())
print('action space:', env.action_space())
print('initial observation:', obs.shape)

# 行動選択と選択した行動の入力
action = 0
obs, r, done = env.step(action)
print('next observation:', obs.shape) # 環境から返ってくる次状態
print('reward:', r)
print('done:', done)
env.render()

## ネットワーク構造
ネットワークモデルを定義します．
ここでは，環境からの観測値が数値情報の場合と，画像情報の場合，それぞれにおけるネットワークを定義しています．

数値情報の場合は全結合層4層とし，画像情報の場合は畳み込み層3層と全結合層2層としています．


In [ ]:
# 観測値が数値情報の場合
class DQN_continuous(nn.Module):
    def __init__(self, input_shape, n_actions):
        super(DQN_continuous, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_shape, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, n_actions)
        )

    def forward(self, x):
        return self.fc(x)

In [ ]:
# 観測値が画像情報の場合
class DQN_image(nn.Module):
    def __init__(self, input_shape, n_actions):
        super(DQN_image, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(input_shape[0], 32, kernel_size=8, stride=4),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1),
            nn.ReLU()
        )

        conv_out_size = self._get_conv_out(input_shape)
        self.fc = nn.Sequential(
            nn.Linear(conv_out_size, 512),
            nn.ReLU(),
            nn.Linear(512, n_actions)
        )

    def _get_conv_out(self, shape):
        o = self.conv(torch.zeros(1, *shape))
        return int(np.prod(o.size()))

    def forward(self, x):
        conv_out = self.conv(x).view(x.size()[0], -1)
        return self.fc(conv_out)

## Deep Q-Networkにおける学習工夫の定義

Deep Q-Networkでは，学習の促進と安定化の為に，いくつか工夫を施して学習を行っています．
代表的な工夫として，Experience Replay, Target Q-Network, Reward Clippingと呼ばれる3つの工夫があります．


### Experience Replay

DQNでは，獲得した経験を直接使用し学習するのではなく，獲得した経験を一度Replay Bufferと呼ばれるBufferに格納し，学習する際にはBufferから経験をランダムに取得することにより学習を行います．
これをExperience Replayと呼び，データの再利用を行うことで，データ効率を高め効率的な学習を行います．

Bufferへは，「現在の状態，その時に選択された行動，行動によって遷移した状態（次状態），その時の報酬」の4種類の情報を1つの経験として蓄積します．\
まず，`Experience`という変数を定義します．
ここでは，`state`, `action`, `reward`, `done`, `next_state`が1セットとなるようなデータ構造（辞書オブジェクト）を定義します．\
その後，Experience Bufferクラスを定義します．
Experience Bufferクラスでは，経験を蓄積する`buffer`（`capacity = buffer`へ格納する経験の数）を定義します．
append関数では，メモリへ経験を格納します．
また，sample関数では，指定したバッチサイズ (`batch_size`) 分の経験をランダムに選択し，返す関数を定義します．

In [ ]:
Experience = collections.namedtuple('Experience', field_names=['state', 'action', 'reward', 'done', 'new_state'])

class ExperienceBuffer:
    def __init__(self, capacity):
        self.buffer = collections.deque(maxlen=capacity)

    def __len__(self):
        return len(self.buffer)

    def append(self, experience):
        self.buffer.append(experience)

    def sample(self, batch_size):
        indices = np.random.choice(len(self.buffer), batch_size, replace=False)
        states, actions, rewards, dones, next_states = zip(*[self.buffer[idx] for idx in indices])
        return np.array(states), np.array(actions), np.array(rewards, dtype=np.float32), \
               np.array(dones, dtype=np.uint8), np.array(next_states)

### Target Q-Network

DQNで誤差を算出する際，目標値として用いる行動価値関数と，現在の行動価値関数をそれぞれ別のネットワークの出力を用いて誤差を算出します．\
目標値として算出に用いるネットワークは，一定周期経過するまで重みを固定したネットワークとし，一定周期で現在のネットワークと同期しながら学習を行います．
この工夫をTarget Q-Networkと呼び，学習の安定化を図ります．

`target_net`を現在の`net`と同期する`sync_network`を定義します．

In [ ]:
def sync_network():
    tgt_net.load_state_dict(net.state_dict())

### Reward clipping

DQNでは，学習に用いる報酬値を，以下の通りにクリッピングします．
- 報酬値が正：+1
- 報酬値が0：0
- 報酬値が負：-1

これにより，学習における報酬の外れ値に対する過剰反応を防ぐことができます.

In [ ]:
def clipreward(reward):
  """Bin reward to {+1, 0, -1} by its sign."""
  return np.sign(reward)

## エージェントの定義

エージェントが，環境に対して行動価値に沿った行動を選択し，環境から経験を取得，Experience Bufferへ獲得した経験を記録するようにします．

エージェントの環境に対する動きのクラスを定義します．\
play_step関数は，環境に対し行動を決定する関数です．
$\epsilon$-greedy法を用いて，一定の割合でランダムに行動選択を行います．
それ以外の場合は，ネットワークへ環境情報（画像）を入力し，行動を決定します．

In [ ]:
class Agent:
    def __init__(self, env, exp_buffer):
        self.env = env
        self.exp_buffer = exp_buffer
        self._reset()

    def _reset(self):
        self.state = env.reset()
        self.total_reward = 0.0

    def play_step(self, net=None, epsilon=0.0):
        done_reward = None

        if np.random.random() < epsilon:
            action = env.random_action()
        else:
            state_a = np.array([self.state], copy=False)
            state_v = torch.tensor(state_a).float()
            if use_cuda:
              state_v = state_v.cuda()
            q_vals_v = net(state_v)
            _, act_v = torch.max(q_vals_v, dim=1)
            action = int(act_v.item())

        # do step in the environment
        new_state, reward, is_done = self.env.step(action)
        # reward = clipreward(reward)
        self.total_reward += reward

        exp = Experience(self.state, action, reward, is_done, new_state)
        self.exp_buffer.append(exp)
        self.state = new_state
        if is_done:
            done_reward = self.total_reward
            self._reset()
        return done_reward

    def play_random_step(self):
        action = env.random_action()

        # do step in the environment
        new_state, reward, is_done = self.env.step(action)
        # reward = clipreward(reward)

        exp = Experience(self.state, action, reward, is_done, new_state)
        self.exp_buffer.append(exp)
        self.state = new_state
        if is_done:
            self._reset()
        return

## TD誤差の計算


DQNでは，TD誤差と呼ばれる次状態の推定価値と，実際に選択した行動から得られる価値の差を用いて学習を行います．
この時，次状態の推定価値は教師あり学習の教師と同じ役割を持ちます．\
DQNは，Q学習をもとにしているため，現在の行動価値関数を最適行動価値関数になるように更新を行っていきます．

TD誤差の計算を行う関数を定義します．\
calc_loss関数では，`replay_buffer`からランダムに取得した`batch`分の経験をもとに，以下のLoss計算を行います．

$$
L_{\theta}=\frac{1}{2}(r+\gamma \max_{a'}Q_{\theta_{i}}(s',a')-Q_{\theta_{i}}(s,a))^{2}
$$

In [ ]:
def calc_loss(batch, net, tgt_net):
    states, actions, rewards, dones, next_states = batch

    states_v = torch.tensor(states).float()
    next_states_v = torch.tensor(next_states).float()
    actions_v = torch.tensor(actions)
    rewards_v = torch.tensor(rewards)
    done_mask = torch.BoolTensor(dones)
    if use_cuda:
      states_v = states_v.cuda()
      next_states_v = next_states_v.cuda()
      actions_v = actions_v.cuda()
      rewards_v = rewards_v.cuda()
      done_mask = done_mask.cuda()

    state_action_values = net(states_v).gather(1, actions_v.unsqueeze(-1)).squeeze(-1)
    next_state_values = tgt_net(next_states_v).max(1)[0]
    next_state_values[done_mask] = 0.0
    next_state_values = next_state_values.detach()

    expected_state_action_values = next_state_values * GAMMA + rewards_v
    return nn.MSELoss()(state_action_values, expected_state_action_values)

## 学習
DQNを用いて学習を行います．

学習にはそれなりの時間がかかります．
そのため，今回は学習途中のモデルをロードし，途中から学習を行います．

まず，環境を初期化し，経験をReplayBufferへ蓄積します．
十分に蓄積された後，パラメータの更新を行います．
また，`SNC_TARGET_FRAMES`で指定した回数ごとに，`target_net`のパラメータを`net`のパラメータと同じになるように同期を行います．

In [ ]:
# 学習途中モデルのダウンロード
!wget -q http://mprg.cs.chubu.ac.jp/~itaya/share/mprg_colab/DQN_application/models.zip
!unzip -q -o models.zip

In [ ]:
# 学習時のハイパーパラメータ
GAMMA = 0.99
BATCH_SIZE = 32
LEARNING_RATE = 1e-4
SYNC_TARGET_FRAMES = 1000
REPLAY_SIZE = 10000
REPLAY_START_SIZE = 10000

# ε-greedyの設定
EPSILON_DECAY_LAST_FRAME = 300000
EPSILON_START = 1.0
EPSILON_FINAL = 0.02

# 環境の観測値を数値情報とするか画像とするか　(num：数値情報， image:画像情報)
INPUT = "num"

#　学習済みモデルを使用するか (250,000ステップ学習したモデルをロード)
LOAD_MODEL = "models/wave-num-premodel.dat"

# 何フレーム(ステップ)学習するか
num_frame = 500000

In [ ]:
env = ENV(obs_mode=INPUT) # 環境の構築
env.seed(seed)

frame_idx = 0
buffer = ExperienceBuffer(REPLAY_SIZE) # Bufferの構築
agent = Agent(env, buffer) # エージェントの構築
epsilon = EPSILON_START
record_reward = []
record_step = []

Experience = collections.namedtuple('Experience', field_names=['state', 'action', 'reward', 'done', 'new_state'])
if INPUT == "num":
  net = DQN_continuous(env.observation_space(), env.action_space())
  tgt_net = DQN_continuous(env.observation_space(), env.action_space())
elif INPUT == "image":
  net = DQN_image(env.observation_space(), env.action_space())
  tgt_net = DQN_image(env.observation_space(), env.action_space())
if use_cuda:
  net = net.cuda()
  tgt_net = tgt_net.cuda()

# 最適化手法
optimizer = optim.RMSprop(net.parameters(), lr=LEARNING_RATE)

# 学習途中モデルのロード
if LOAD_MODEL != "":
  checkpoint = torch.load(LOAD_MODEL)
  net.load_state_dict(checkpoint['model_state_dict'])
  tgt_net.load_state_dict(checkpoint['model_state_dict'])
  epsilon = checkpoint['epsilon']
  frame_idx = checkpoint['frame_idx']
  record_reward = checkpoint['record_reward']
  record_step = checkpoint['record_step']
  optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
  print("load model: {}".format(LOAD_MODEL))
else:
  print("Not model loading")


total_rewards = []
best_mean_reward = None

# 空のBefferにある程度の経験を収集
print("Collect experience....")
while len(buffer) < REPLAY_START_SIZE:
  if LOAD_MODEL == "":
    agent.play_random_step()
  else:
    agent.play_step(net, epsilon=0.0)
agent._reset()
print("Filled buffer {}".format(len(buffer)))

# エージェントの学習開始
print("Leaning start")
ts = time.time()
while frame_idx < num_frame:
    frame_idx += 1
    epsilon = max(EPSILON_FINAL, EPSILON_START - frame_idx / EPSILON_DECAY_LAST_FRAME)

    reward = agent.play_step(net, epsilon) # 環境の1step
    if reward is not None:
        total_rewards.append(reward)
        mean_reward = np.mean(total_rewards[-100:])
        print("Frame {0}/{1}: episode {2}, reward {3:.3f}, mean reward {4:.3f}, eps {5:.2f}, time {6}".format(frame_idx, num_frame, len(total_rewards), reward, mean_reward, epsilon, datetime.timedelta(seconds = time.time() - ts)))
        record_reward.append(mean_reward)
        record_step.append(frame_idx)

        if best_mean_reward is None or best_mean_reward < mean_reward:
            torch.save(
                {'epsilon':epsilon, 'frame_idx':frame_idx, 'model_state_dict':net.state_dict(), 'optimizer_state_dict':optimizer.state_dict()},
                "wave-{}-best.dat".format(INPUT))
            if best_mean_reward is not None:
                print("Best mean reward updated {0:.3f} -> {1:.3f}, model saved".format(best_mean_reward, mean_reward))
            best_mean_reward = mean_reward

    # target networkの同期
    if (frame_idx % SYNC_TARGET_FRAMES) == 0:
        sync_network()

    optimizer.zero_grad()
    batch = buffer.sample(BATCH_SIZE)
    loss_t = calc_loss(batch, net, tgt_net) # 誤差計算
    loss_t.backward()
    optimizer.step()

## 学習時の平均スコアの推移
横軸エピソード数，縦軸平均スコアとしたグラフを描画してみます．

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure()
plt.plot(record_step, record_reward, color="red")
plt.grid()
plt.xlabel("step")
plt.ylabel("mean reward")
plt.savefig("./dqn_step_per_reward_{}.png".format(INPUT))
plt.show()
plt.clf()
plt.close()

グラフのダウンロード

In [ ]:
from google.colab import files
files.download("./dqn_step_per_reward_{}.png".format(INPUT))

## 評価
学習したネットワーク（エージェント）を確認してみます．

ここでは，`frames`に描画したフレームを順次格納します．

In [ ]:
frames = []
for i in range(1):
    state = env.reset()
    done = False
    t = 0
    rewards = 0.0

    while not done:
        _, img = env.render(mode='rgb_array')
        frames.append(img)
        state_a = np.array([state], copy=False)
        state_v = torch.tensor(state_a).float()
        if use_cuda:
          state_v = state_v.cuda()
        q_vals_v = net(state_v)
        _, act_v = torch.max(q_vals_v, dim=1)
        action = int(act_v.item())
        new_state, reward, done = env.step(action)
        rewards += reward
        state = new_state
    print("reward: ", rewards)

## 描画

maptlotlibを用いて，保存した動画フレームをアニメーションとして作成し，表示しています．

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML


# フレーム画像をアニメーションに変換
def video_anime(imgs):
    fig = plt.figure(figsize=(imgs[0].shape[1] / 72.0, imgs[0].shape[0] / 72.0), dpi = 72)  # 表示サイズ指定

    mov = []
    for i in range(len(imgs)):  # フレームを1枚づつmovにアペンド
        img = plt.imshow(imgs[i], animated=True)
        plt.axis('off')
        mov.append([img])

    # アニメーション作成
    anime = animation.ArtistAnimation(fig, mov, interval=200, repeat_delay=0, repeat=False)
    plt.close()
    return anime

# HTML5でアニメーションをインラインに動画表示
HTML(video_anime(frames).to_html5_video())

## 観測情報が画像の場合 (学習済みモデルでの評価)

環境とネットワークの構築

In [ ]:
INPUT = "image"
LOAD_MODEL = "models/wave-image-trainmodel.dat"

env = ENV(obs_mode=INPUT)
env.seed(seed)

net = DQN_image(env.observation_space(), env.action_space())
if use_cuda:
  net = net.cuda()

if LOAD_MODEL != "":
  checkpoint = torch.load(LOAD_MODEL)
  net.load_state_dict(checkpoint['model_state_dict'])
  record_reward = checkpoint['record_reward']
  record_step = checkpoint['record_step']
  print("load model: {}".format(LOAD_MODEL))
else:
  print("Not model loading")

平均スコアの推移

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure()
plt.plot(record_step, record_reward, color="red")
plt.grid()
plt.xlabel("step")
plt.ylabel("mean reward")
plt.savefig("./dqn_step_per_reward_{}.png".format(INPUT))
plt.show()
plt.clf()
plt.close()

グラフのダウンロード

In [ ]:
from google.colab import files
files.download("./dqn_step_per_reward_{}.png".format(INPUT))

評価と描画

In [ ]:
frames = []
for i in range(1):
    state = env.reset()
    done = False
    t = 0
    rewards = 0.0

    while not done:
        _, img = env.render(mode='rgb_array')
        frames.append(img)
        state_a = np.array([state], copy=False)
        state_v = torch.tensor(state_a).float()
        if use_cuda:
          state_v = state_v.cuda()
        q_vals_v = net(state_v)
        _, act_v = torch.max(q_vals_v, dim=1)
        action = int(act_v.item())
        new_state, reward, done = env.step(action)
        rewards += reward
        state = new_state
    print(rewards)


# 実行結果の表示
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

def video_anime(imgs):
    fig = plt.figure(figsize=(imgs[0].shape[1] / 72.0, imgs[0].shape[0] / 72.0), dpi = 72)
    mov = []
    for i in range(len(imgs)):
        img = plt.imshow(imgs[i], animated=True)
        plt.axis('off')
        mov.append([img])
    anime = animation.ArtistAnimation(fig, mov, interval=200, repeat_delay=0, repeat=False)
    plt.close()
    return anime

# HTML5でアニメーションをインラインに動画表示
HTML(video_anime(frames).to_html5_video())

# 課題

1. 環境の設定（行動リスト，報酬設計など）を変更し学習の様子を確認してみましょう

  **ヒント**
  *   各パラメータの上下限設定（WAVEクラスinit関数内）
  *   波形の初期化方法（WAVEクラスinit_wave関数内）
  *   行動の設定（ENVクラスinit関数内）
  *   目標波形の設定（ENVクラスinit関数内）
  *   報酬設計（ENVクラスcal_reward関数内）
  *   観測情報の設計（ENVクラスstep関数内）


